# Which borough has the most noise complaints (or, more selecting data)

## Install

In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
!pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="pandas-test-domain-3", port="8073", reset=True)

# Data owner: upload data

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# The usual preamble
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from syft import Worker, MessageStatus
from syft.service.project.project import Project
from syft.util.util import PANDAS_DATA

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

plt.rcParams['figure.figsize'] = (15, 5)

## Load data

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from NYC Open Data.

In [ ]:
# because of mixed types we specify dtype to prevent any errors
complaints = pd.read_csv(sy.autocache(f"{PANDAS_DATA}/311-service-requests.csv"), dtype='unicode')

Depending on your pandas version, you might see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

In [ ]:
complaints

## Create Mock data

In [ ]:
from syft.serde.mock import CachedFaker
from random import randint
import random

Let's create the mock data for the complaint dataset.

In [ ]:
fake = CachedFaker()

In [ ]:
fake_functions = {
    "Unique Key": lambda x: randint(1,1000000),
    "Location": lambda x: (fake.coordinate(), fake.coordinate()),
    "Agency": lambda x: random.choice(["NYPD", "DOHMH", "DPR"]),
    "X Coordinate (State Plane)": lambda x: randint(1,1000000),
    "Y Coordinate (State Plane)": lambda x: randint(1,1000000),
    "Complaint Type": lambda x: random.choice(["Illegal Parking", "Noise - Street/Sidewalk", "'Animal in a Park'"]),
    "Descriptor": lambda x: random.choice([ 'Branch or Limb Has Fallen Down','Branches Damaged','Broken Fence', 'Broken Glass']),
    "School Number": lambda x: random.choice([ 'B073', 'B077', 'B079', 'B080-01', 'B087', 'B099', 'B100', 'B102', 'B109', 'B111']),
    "Bridge Highway Segment": lambda x: random.choice([ 'Grand Central Pkwy (Exit 1 E-W)',
                                                       'Grand Central Pkwy (Exit 10) - 69th Rd-Jewel Ave (Exit 11)',
                                                       'GrandCentral Pkwy/VanWyck Expwy/College Point Blvd (Exit 22 A-E)',
                                                       'Hamilton Ave (Exit 2A) - Gowanus Expwy (I-278) (Exit 1)',
                                                       'Harding Ave (Exit 9) - Throgs Neck Br'])
}

In [ ]:
fake_triggers = {
    "Street": lambda x : fake.street_name(),
    "Date": lambda x : fake.date_time(),
    "Long": lambda x : fake.coordinate(),
    "Lat": lambda x : fake.coordinate(),
    "Address": lambda x : fake.address(),
    "Name": lambda x : fake.name(),
    "City": lambda x : fake.city(),
    "Zip": lambda x : fake.zipcode(),
}

In [ ]:
mock_data = dict()
for col in complaints.columns:
    col_vals = complaints[col]
    
    if col in fake_functions:
        mock_func = fake_functions[col]
    elif len(set(complaints[col])) < 100:
        values = list(set(complaints[col]))
        mock_func = lambda x: random.choice(values)
    else:
        for trigger in fake_triggers.keys():
            if trigger in col:
                mock_func = fake_triggers[trigger]
    mock_data[col] = [mock_func(None) for x in range(len(complaints))]

In [ ]:
mock = pd.DataFrame(data=mock_data)

In [ ]:
mock.head()

In [ ]:
dataset = sy.Dataset(name="bikes",
                     asset_list=[sy.Asset(name="complaints", data=complaints, mock=mock, mock_is_real=False)])
domain_client.upload_dataset(dataset)

## Create data scientist

In [ ]:
user = domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

# todo: give user data scientist role

guest_domain_client = node.client

guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: Create result pointer

## Download mock and request pointer

### Get mock

In [ ]:
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
ds = guest_domain_client.datasets[0]

In [ ]:
asset = ds.assets["complaints"]

In [ ]:
complaints_pointer = asset.mock

### Selecting only noise complaints


I'd like to know which borough has the most noise complaints. First, we'll take a look at the data to see what it looks like:

In [ ]:
complaints_pointer[:5]

To get the noise complaints, we need to find the rows where the "Complaint Type" column is "Noise - Street/Sidewalk". I'll show you how to do that, and then explain what's going on.



In [ ]:
noise_complaints_pointer = complaints_pointer[complaints_pointer['Complaint Type'] == "Noise - Street/Sidewalk"]
noise_complaints_pointer[:3]

If you look at noise_complaints, you'll see that this worked, and it only contains complaints with the right complaint type. But how does this work? Let's deconstruct it into two pieces

In [ ]:
complaints_pointer['Complaint Type'] == "Noise - Street/Sidewalk"

This is a big array of Trues and Falses, one for each row in our dataframe. When we index our dataframe with this array, we get just the rows where our boolean array evaluated to True. It's important to note that for row filtering by a boolean array the length of our dataframe's index must be the same length as the boolean array used for filtering.

You can also combine more than one condition with the & operator like this:

In [ ]:
is_noise = (complaints_pointer['Complaint Type'] == "Noise - Street/Sidewalk")
in_brooklyn = complaints_pointer['Borough'] == "BROOKLYN"
complaints_pointer[is_noise & in_brooklyn][:5]

Or if we just wanted a few columns:



In [ ]:
complaints_pointer[is_noise & in_brooklyn][['Complaint Type', 'Borough', 'Created Date', 'Descriptor']][:10]

### A digression about numpy arrays

On the inside, the type of a column is pd.Series

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.Series([1,2,3])

and pandas Series are internally numpy arrays. If you add .values to the end of any Series, you'll get its internal numpy array

In [ ]:
np.array([1,2,3])

In [ ]:
pd.Series([1,2,3]).values

So this binary-array-selection business is actually something that works with any numpy array:

In [ ]:
arr = np.array([1,2,3])

In [ ]:
arr != 2

In [ ]:
arr[arr != 2]

### So, which borough has the most noise complaints?

In [ ]:
is_noise = complaints_pointer['Complaint Type'] == "Noise - Street/Sidewalk"
noise_complaints_pointer = complaints_pointer[is_noise]
noise_complaints_pointer['Borough'].value_counts()

It's the BRONX (for this Mock)! But what if we wanted to divide by the total number of complaints, to make it make a bit more sense? That would be easy too:

In [ ]:
noise_complaint_counts_pointer = noise_complaints_pointer['Borough'].value_counts()
complaint_counts_pointer = complaints_pointer['Borough'].value_counts()

In [ ]:
noise_complaint_counts_pointer

In [ ]:
noise_complaint_counts_pointer / complaint_counts_pointer

Oops, why was that zero? That's no good. This is because of integer division in Python 2. Let's fix it, by converting complaint_counts into an array of floats.

In [ ]:
noise_complaint_counts_pointer / complaint_counts_pointer.astype(float)

In [ ]:
(noise_complaint_counts_pointer / complaint_counts_pointer.astype(float)).plot(kind='bar')

So Bronx really does complain more about noise than the other boroughs in our mock! Neat.

In [ ]:
res_ptr = noise_complaint_counts_pointer / complaint_counts_pointer.astype(float)

## Request real result

Now that we finished our analysis on the pointers, we can request the real result.

In [ ]:
res_ptr.request(guest_client)

# Data owner: approve request

In [ ]:
domain_client = node.client.login(email="info@openmined.org", password="changethis")
domain_client.api.services.request[0].approve()

# Data scientist: fetch result

In [ ]:
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")


In [ ]:
real_result = res_ptr.get_from(guest_client)

In [ ]:
real_result[:10].plot(kind="bar")